Wow! Here's where we introduce people to our Kaggle document. You might need to click the large black button in the upper right-hand corner that says Save and Edit in order to get all the functionality out of this document. You'll need to run each cell in order. To run a cell, select it and press Shift+Enter simultaneously.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt # plotting library

from scipy.stats import beta as betafunction # Just want to try something...

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

By the way, these are markdown cells – they're basically just presentable text. We can use them to tell things to our readers: we can give them instructions, or tell them about the log-logistic distribution. First, we need to run a bunch of cells to initialize our plotting.

In [ ]:
#Takes in an array x, and alpha, beta parameter values
def log_logistic_test_PDF(x, alpha, beta):
    #The idiot loop
    if beta <=0:
        print("Beta is either 0, or less than 0 and that's not allowed. Stop it.")
        return False
    if alpha <=0:
        print("The funciton is undefined for values of alpha that are less than 0, or equal to 0. I'm going to have to ask you to stop.")
        return False
    
    pdf = []
    for i in x:
        if i<0:
            print("Warning: cannot compute a negative probability")
        pdf.append(log_logistic_evalPDF(i, alpha, beta))
    return pdf

In [ ]:
#Takes in an array x, and alpha, beta parameter values
def log_logistic_test_CDF(x, alpha, beta):
    #The idiot loop
    if beta <=0:
        print("Beta is either 0, or less than 0 and that's not allowed. Stop it.")
        return False
    if alpha <=0:
        print("The funciton is undefined for values of alpha that are less than 0, or equal to 0. I'm going to have to ask you to stop.")
        return False
    
    cdf = []
    for i in x:
        if i<0:
            print("Warning: cannot compute a negative probability")
        cdf.append(log_logistic_evalCDF(i, alpha, beta))
    return cdf

In [ ]:
#Evaluates the PDF at a point, x
def log_logistic_evalPDF(x, alpha, beta):
    if beta >=1:
        val = ((beta/alpha)*(x/alpha)**(beta-1))/((1+(x/alpha)**beta)**2)
    elif (beta > 0 and beta < 1):
        # Treat discontinuities at 0
        if(x==0):
            return np.inf
        else:
            try:
                power = 1-beta
                val = ((beta/alpha))/((x/alpha)**(power)*(1+(x/alpha)**beta)**2)
            except Error as err:
                print("Caught an error: {0}. Returning infinity.".format(err))
                return np.inf
    return val

In [ ]:
#Evaluates the CDF at a point, x
def log_logistic_evalCDF(x, alpha, beta): 
    if beta==0:
        print("Error, beta cannot be equal to 0.")
        return
    elif beta >= 1: 
        val=(x**beta)/((alpha**beta)+(x**beta))
    else: # For beta between 0 and 1
        if (x==0): # Account for numpy's weird behaviour at 0
            return 0
        else:
            val=(x**beta)/((alpha**beta)+(x**beta))
    return val

In [ ]:
# Import all the interaction widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def moment(alpha, beta, n):
    try:
        if(beta > n):
            return alpha**n*(n*np.pi/beta)/(np.sin(n*np.pi/beta))
        else:
            return "Undefined"
    except Exception as err:
        return "Undefined"
    
# Not sure yet if I can generate arbitrary moments, or just one at a time
def moment_generator(alpha, beta):
    '''
    # The distribution is not unimodal for beta<=1, so moments are undefined
    if(beta<=1):
        return "Undefined", "Undefined" # No, this is not how you play the game!
    mean = (alpha*np.pi/beta)/(np.sin(np.pi/beta))
    if (beta<=2):
        return np.around(mean, 3), "Undefined" # Standard deviation is undefined for beta less than 2
    stdev = np.sqrt((alpha**2)*(((2*np.pi*beta)/(np.sin(2*np.pi/beta)))-((np.pi/beta)/(np.sin(np.pi/beta)))))
    
    print("OG: {0}, {1}".format(mean, stdev))
    '''
    # From our four moments about the origin...
    mu1=moment(alpha, beta, 1)
    mu2=moment(alpha, beta, 2)
    mu3=moment(alpha, beta, 3)
    mu4=moment(alpha, beta, 4)
    
    #... we can compute the moments about the mean
    mean = mu1
    stdev = (mu2 - mu1**2)**(1/2)
    skewness = (mu3 - 3*mu1*mu2 + 2*mu1**2)/(stdev**3)
    kurtosis = (mu4 - 4*mu1*mu3 + 6*mu1**2*mu2 - 3*mu1**4)/(stdev**4)-3
    
    return np.around(mean, 3), np.around(stdev, 3), np.around(skewness, 3), np.around(kurtosis, 3)

In [ ]:
moment_generator(1,2)

The log-logistic distribution is (blah blah blah). It is described by two parameters, and (blah blah blah). (blah blah blah) the (blah blah blah) to (blah blah blah) and then (blah blah blah) so that (blah blah blah) can (blah blah blah). Think about which parameter affects skewness, and which parameter affects Kurtosis (note: both parameters affect both quantities, but one dominates over the other in each).

Conveniently, our shape parameter beta gives the median of this distribution.

Notice that the mean is undefined for beta less than 1 – why do you think that is?
More subtly, the standard deviation is undefined for beta less than 2. 

You can change the values with the sliders, or just input the values you desire in the textbox beside the sliders. Explore what happens to the distribution as each parameter becomes large or small.

In [ ]:
# We will need to make a separate method for the Fisk distribution
def log_logistic_interactionPDF(alpha, beta):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    #Concatenate two arrays to improve resolution near 0
    x = np.concatenate((np.linspace(0,0.1,10), np.linspace(0.1,10,100)), axis=None)
    pdf = log_logistic_test_PDF(x, alpha, beta)
    
    ax.plot(x, pdf)
    
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim([0, 10])
    plt.ylim([0, 2])
    plt.title("Probability Distribution")
    
    moments = moment_generator(alpha, beta)
    plt.text(6, 1.8, "Mean = {0}".format(moments[0]))
    plt.text(6, 1.6, "Standard Deviation = {0}".format(moments[1]))
    fig.patch.set_facecolor('white') # we want to see the axis values in dark mode
    #plt.plot(x, pdf)
    #plt.yscale("log") # The log scale makes it difficult to interpret these graphs...

In [ ]:
interact(log_logistic_interactionPDF, alpha=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1), beta=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1))

Now that you've found things, I'm just going to tell you what you should have found, for good measure! Beta dominates the effects of skewness, while alpha dominates the kurtosis of this distribution.

We can also look at the cumulative distribution. Once again, run the cells below and explore the limiting cases of each parameter.

In [ ]:
# Again, make a separate method later for the Fisk distribution
def log_logistic_interactionCDF(alpha, beta):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    #Concatenate two arrays to improve resolution near 0
    x = np.concatenate((np.linspace(0,0.1,100), np.linspace(0.1,10,100)), axis=None)
    pdf = log_logistic_test_CDF(x, alpha, beta)
    
    ax.plot(x, pdf)
    
    plt.xlabel("x")
    plt.ylabel("F(x)")
    plt.xlim([0, 10])
    plt.ylim([0, 2])
    plt.title("Cumulative Distribution")
    fig.patch.set_facecolor('white') # we want to see the axis values in dark mode
    #plt.plot(x, pdf)
    #plt.yscale("log")

In [ ]:
interact(log_logistic_interactionCDF, alpha=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1), beta=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1))

Why not look at both side by side?

In [ ]:
def log_logistic_double_display(alpha, beta):
    fig = plt.figure(figsize=(12, 6))
    
    #Concatenate two arrays to improve resolution near 0
    x = np.concatenate((np.linspace(0,0.1,10), np.linspace(0.1,5,50)), axis=None)
    pdf = log_logistic_test_PDF(x, alpha, beta)
    cdf = log_logistic_test_CDF(x, alpha, beta)
    
    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    ax.plot(x, pdf)
    ax2.plot(x, cdf)
    
    ax.set_xlabel("x")
    ax.set_ylabel("f(x)")
    ax2.set_xlabel("x")
    ax2.set_ylabel("f(x)")
    
    ax.set_xlim([0, 5])
    ax.set_ylim([0, 2])
    ax.set_title("Probability Distribution")
    ax2.set_xlim([0, 5])
    ax2.set_ylim([0, 2])
    ax2.set_title("Cumulative Distribution")
    
    fig.patch.set_facecolor('white') # we want to see the axis values in dark mode

In [ ]:
interact(log_logistic_double_display, alpha=widgets.FloatSlider(min=0.1, max=10, step=0.2, value=1), beta=widgets.FloatSlider(min=0.1, max=10, step=0.2, value=1))

There's no lesson plan yet... but with a few tweaks, I wonder if I can make skewness and curtosis sliders rather than alpha and beta sliders. After letting them play with those, we can hand them the alpha and beta sliders and let them play with those, which might give them a better understanding of what alpha and beta mean. Or, this might just be too difficult to implement.

Update: Too difficult to implement.

In Economics, the log-logistic distribution is referred to as the Fisk Distribution, and is used to model the distribution of wealth. Our shape parameter (Beta) takes on a new meaning too: The Gini coefficient is defined as G=(1/Beta), and is a measure of wealth dispersion intended to measure the magnitude of income inequality in a country [Cite something other than Wikipedia... perhaps OECD?]. A Gini coefficient of 0 denotes perfect equality, while a coefficient of 1 indicates complete inequality. Since G=(1/Beta), we can only have perfect equality in the limit as Beta tends to infinity, wheras complete inequality occurs when Beta equals 1.

Let's look at some data now. I'll explain what this is when I'm finished coding.

In [ ]:
# HEY! It turns out that other people can't read these files.
# If you can't read this data, for now just keep running cells. I've hardcoded
# the values later...
pandaData = pd.read_csv("../input/various-wealth-distributions/dfa-income-levels.csv")

pandaData

In [ ]:
# Need to interpret our income brackets as x-values...
incbrak = [0.995, 0.895, .70, .50, .30, .10]
#incbrak = [0.1, 0.3, 0.5, 0.7, 0.895, 0.995]
#incbrak = [0.1, 0.3, 0.5, 0.7, 0.9]

# We want the top 1 percent to be at 0
#revincbrak = [0.005, 0.105, 0.3, 0.5, 0.7, 0.9]
#print(revincbrak)

#revincbins = [[0.0,0.01],[0.01,0.2],[0.2,0.4],[0.4,0.6],[0.6,0.8],[0.8,1.0]]
yrqrtr = 110
print(pandaData['Date'][yrqrtr*6])
nworth = []
for i in range(0, 6):
    nworth.append(pandaData['Net worth'][i+yrqrtr*len(incbrak)])
print(nworth)

#revnworth = []
#for i in range(0, len(nworth)):
#    revnworth.append(nworth[len(nworth)-i-1])
#print(revnworth)
    
# Establish the cumulative net worth by summing over all the networth bins
cumnetworth = [nworth[0]]
for i in range(1, len(nworth)):
    cumnetworth.append(cumnetworth[i-1]+nworth[i])
#cumnetworth[len(cumnetworth)] = cumnetworth[len(cumnetworth)-1] + revnworth[len(revnworth)-1]
print(cumnetworth)

# We need to normalize the cumulative net worth – express as a fraction of
# the total net worth
cumnetworth = cumnetworth/max(cumnetworth)
print(cumnetworth)

In [ ]:
# If you can't load the data, run this cell!
#incbrak = [0.1, 0.3, 0.5, 0.7, 0.9]
#cumnetworth = [0.08339383, 0.23661461, 0.70103327, 0.92135397, 1.]
incbrak = [0.1, 0.3, 0.5, 0.7, 0.895, 0.995]
#cumnetworth = [0.04896001, 0.07864603, 0.29896673, 0.76338539, 0.91660617, 1.        ]
cumnetworth = [0.24681051, 0.71378521, 0.85827444, 0.9341023,  0.97637864, 1.]

Note to self: Here, it might be nice to have some plots showing the distributions of wealth. Kinda like those questions in Gabe's report, where he asks the reader to attempt to sketch the CDF and the PDF. Except instead of having the reader attempt to sketch it, we just make the reader look at some histograms...

In [ ]:
def chi2(expected, measured):
    chi2 = 0
    for i in range(0, len(expected)):
        chi2 += (expected[i] - measured[i])**2
    return chi2

In [ ]:
# Initialize some parameters to save the best fit
#bestAlpha = 1
#bestBeta = 1
#bestchi2fit = 100

def update(chi2fit):
    bestchi2fit = chi2fit

# This is the fisk cumulative distribtion function
def fisk_CDF(alpha, beta):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    #Concatenate two arrays to improve resolution near 0
    x = np.concatenate((np.linspace(0,0.1,50), np.linspace(0.1,1,50)), axis=None)
    cdf = log_logistic_test_CDF(x, alpha, beta)
    # Need to evaluate the CDF at each point in the income bracket for our tests
    compare_points = log_logistic_test_CDF(incbrak, alpha, beta)
    
    ax.plot(x, cdf)
    ax.plot(incbrak, cumnetworth, 'r.')
    
    # Record the chi2 fit and also the parameter that yield the best fit.
    chi2fit = chi2(cumnetworth, compare_points)
    '''
    if (chi2fit < bestchi2fit):
        update(chi2fit)
        bestAlpha = alpha
        bestBeta = beta
    
    plt.text(0.1, 0.9, "Best Parameters: ")
    plt.text(0.1, 0.8, "Alpha: {0}".format(alpha))
    plt.text(0.1, 0.7, "Beta: {0}".format(beta))
    plt.text(0.1, 0.6, "Goodness: {0}".format(np.round(bestchi2fit, 3)))
    '''
    plt.xlabel("Fraction of population")
    plt.ylabel("Fraction of Wealth")
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.title("Cumulative Net Worth")
    plt.title("Cumulative Distribution. Current fit: Chi^2 = {0}".format(np.round(chi2fit, 3)))
    
    
    
    fig.patch.set_facecolor('white') # we want to see the axis values in dark mode

In [ ]:
interact(fisk_CDF, alpha=widgets.FloatSlider(min=0.01, max=2, step=0.01, value=1), beta=widgets.FloatSlider(min=1, max=5, step=0.1, value=1))